<a href="https://colab.research.google.com/github/whdid502/stt_model_project/blob/feature_extraction/pcm_to_mfcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

tensorflow 1.13.2 버전 필요

In [ ]:
!pip install tensorflow==1.13.2

     |████████████████████████████████| 92.6MB 71kB/s 
     |████████████████████████████████| 3.2MB 31.7MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 368kB 38.5MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
!pip3 install SpecAugment

In [ ]:
import wave
import librosa
import IPython.display as ipd
# from matplotlib import pyplot as plt
import numpy as np
import argparse
import librosa.display
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from specAugment import spec_augment_tensorflow
from scipy.fftpack import dct
import os

In [ ]:
import matplotlib
matplotlib.use('TKAgg')

pcm to mfcc

In [ ]:
def pcm_to_mfcc(audio_path, noise_injection):

  signal = np.memmap(audio_path, dtype='h', mode='r').astype('float32') # load audio
  data = signal / 32767   # normalize audio

  if noise_injection:           # noise injection
    wn = np.random.randn(len(data))
    data_wn = data + 0.005*wn
  else:
    data_wn = data
  
  sr = 16000
  mel_spectrogram = librosa.feature.melspectrogram(y=data_wn, sr=sr, n_mels=256, hop_length=128, fmax=8000) # data to melspectrogram

  warped_masked_spectrogram = spec_augment_tensorflow.spec_augment(mel_spectrogram=mel_spectrogram) # melspectrogram spec augmentation

  num_ceps = 12
  mfcc = dct(warped_masked_spectrogram, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)] # mel spectrogram to mfcc

  return mfcc

In [ ]:
output = pcm_to_mfcc('/content/drive/My Drive/googledrive/feature_test/KsponSpeech_000001.pcm', True)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
output

array([[-7.49562532e-02, -2.84702599e-01,  1.81425169e-01, ...,
        -5.52867241e-02, -1.92406625e-01,  1.60623193e-01],
       [-2.75385045e-02, -1.22367762e-01,  9.37460214e-02, ...,
        -1.24541800e-02, -1.62427761e-02,  1.17731623e-01],
       [-7.21717775e-02, -1.06115863e-01,  7.93983638e-02, ...,
        -2.44816616e-02,  1.49585530e-01,  2.84625664e-02],
       ...,
       [-3.47786536e-03,  1.18186008e-02,  6.08306611e-03, ...,
        -2.65331077e-03, -4.77150874e-03, -2.84313178e-03],
       [-5.50996093e-03,  9.00761224e-03,  7.85221439e-03, ...,
        -3.16950865e-03, -2.18405272e-03,  3.81596386e-04],
       [-8.14237399e-04,  1.17114745e-02,  1.16785765e-02, ...,
        -2.55373679e-03,  2.33036117e-04, -1.18544069e-03]], dtype=float32)

폴더 훑어서 일괄적용

In [16]:
total_list = []
mfcc_list = []

def mfcc_in_dir(root_dir, batch_size):
    files = os.listdir(root_dir)
    for file in files:
        path = os.path.join(root_dir, file)
        # print(path)
        if os.path.isdir(path):
            mfcc_in_dir(path, 5)
        total_list.append(path)
    file_list_pcm = [file for file in total_list if file.endswith(".pcm")]

    convert_count = 0

    for pcm in file_list_pcm:
      mfcc_list.append(pcm_to_mfcc(pcm, False))
      convert_count += 1
      if convert_count == batch_size:
        print(convert_count)
        break

mfcc_in_dir('/content/drive/My Drive/googledrive/feature_test', 5)

5
5


In [ ]:
mfcc_list

[array([[ 2.92928964e-01, -7.46850967e-01, -5.90744257e-01, ...,
          4.71508563e-01,  8.64344686e-02, -4.33462501e-01],
        [ 2.65936017e-01, -4.30088580e-01, -4.71719921e-01, ...,
          6.50191456e-02, -2.03078054e-02,  6.86875405e-03],
        [ 1.06439635e-01, -2.42870539e-01, -2.51687348e-01, ...,
         -2.06989616e-01, -8.71159732e-02,  2.88361609e-01],
        ...,
        [ 1.74800269e-04, -3.74166702e-04, -4.09646949e-04, ...,
          2.59663735e-04,  1.82679389e-04, -1.28293279e-04],
        [ 8.77332568e-05, -1.56589114e-04, -1.97498011e-04, ...,
          6.16923789e-05,  9.18092119e-05,  1.38410778e-05],
        [ 3.38637547e-05, -7.59008399e-05, -8.09584162e-05, ...,
          5.58791944e-05,  4.38850693e-05, -2.62197009e-05]], dtype=float32),
 array([[-2.80249864e-02, -4.09720689e-01,  1.08020857e-01, ...,
          8.20641965e-02, -3.13689828e-01,  1.14984244e-01],
        [-8.99998844e-02, -1.13807678e+00,  1.41150698e-01, ...,
         -2.43522495e-0